# Monte Carlo Experiment: PSR test vs Non-Central Student’s t-distribution test

**Goal:** For each target Sharpe SR₀, generate a returns-realistic (neg-skew, leptokurtic) mixture of Gaussians whose population Sharpe equals SR₀, and then evaluate PSR vs t-test against that SR₀.

**Data-Generating Process:**
- Mixture body: Normal(μ_core, σ_core); tail: Normal(μ_tail, σ_tail) with prob p_tail (negative shocks).
- To set SR₀: compute the mixture population σ at zero mean, then add a constant μ_shift = SR₀·σ to all observations. (Adding a constant preserves skewness/kurtosis, and sets E[r]/σ = SR₀.)

In [1]:
import numpy as np, pandas as pd
from scipy import stats

# --- settings ---
REPS = 10_000
T = 252*5                       # 5y daily
SR0_annual_list = [0.0, 0.5, 1., 1.5, 2.]
SR0_list = [s/np.sqrt(252) for s in SR0_annual_list]
RSEED = 2025
# Mixture configs: (name, p_tail, mu_tail, sigma_tail, sigma_core)
configs = [
    ("mild",     0.04, -0.03, 0.015, 0.010),
    ("moderate", 0.03, -0.045, 0.020, 0.010),
    ("severe",   0.02, -0.060, 0.025, 0.010),
]

def mixture_variance(p_tail, mu_tail, sigma_tail, mu_core, sigma_core):
    w = 1.0 - p_tail
    mu = w*mu_core + p_tail*mu_tail
    m2 = w*(sigma_core**2 + mu_core**2) + p_tail*(sigma_tail**2 + mu_tail**2)
    return m2 - mu**2

def gen_with_true_SR0(reps, T, cfg, SR0, seed):
    name, p, mu_tail, sig_tail, sig_core = cfg
    # Zero-mean baseline mixture (choose mu_core so mean=0)
    mu_core0 = - p*mu_tail/(1.0 - p)
    std0 = np.sqrt(mixture_variance(p, mu_tail, sig_tail, mu_core0, sig_core))
    mu_shift = SR0 * std0  # sets population Sharpe to SR0, preserves skew/kurt
    rng = np.random.default_rng(seed)
    mask = rng.random((reps, T)) < p
    X = rng.normal(mu_core0 + mu_shift, sig_core, size=(reps, T))
    X[mask] = rng.normal(mu_tail + mu_shift, sig_tail, size=mask.sum())
    return X

def psr_z_T(X, SR0):
    Tn = X.shape[1]
    s = X.std(axis=1, ddof=1)
    sr_hat = X.mean(axis=1)/s
    skew = stats.skew(X, axis=1, bias=False)
    kappa = stats.kurtosis(X, axis=1, fisher=True, bias=False) + 3.0
    den = np.sqrt((1.0/Tn) * (1.0 - skew*SR0 + ((kappa-1.0)/4.0)*(SR0**2)))
    return (sr_hat - SR0)/den

def t_stat(X, SR0):
    Tn = X.shape[1]
    s = X.std(axis=1, ddof=1)
    sr_hat = X.mean(axis=1)/s
    return np.sqrt(Tn)*(sr_hat - SR0)

rows = []
for cfg in configs:
    for SR0 in SR0_list:
        X = gen_with_true_SR0(REPS, T, cfg, SR0, seed=RSEED + int(1e6*SR0) + hash(cfg[0])%10000)
        # realized moments
        avg_skew = float(np.mean(stats.skew(X, axis=1, bias=False)))
        avg_exk  = float(np.mean(stats.kurtosis(X, axis=1, fisher=True, bias=False)))
        # stats and KS
        z = psr_z_T(X, SR0)
        t = t_stat(X, SR0)
        # stats and KS (probability space)
        psr = stats.norm.cdf(z)
        ks_psr = stats.kstest(psr, 'uniform')
        p_t = stats.t.sf(t, df=T-1) if SR0 == 0 else stats.nct.sf(t, df=T-1, nc=np.sqrt(T)*SR0)
        ks_t   = stats.kstest(p_t, 'uniform')
        rows.append({
            'config': cfg[0], 'T': T,
            'SR0_annual': SR0*np.sqrt(252),
            'avg_skew': avg_skew, 'avg_excess_kurtosis': avg_exk,
            'KS_PSR_D': float(ks_psr.statistic), 'KS_t_D': float(ks_t.statistic),
            'PSR_better?': float(ks_psr.statistic) < float(ks_t.statistic),
        })
df = pd.DataFrame(rows).round(6)
df.to_csv('appendix_1.csv')
df

,config,T,SR0_annual,avg_skew,avg_excess_kurtosis,KS_PSR_D,KS_t_D,PSR_better?
0,mild,1260,0.0,-0.890751,2.717294,0.008216,0.008305,True
1,mild,1260,0.5,-0.889079,2.710900,0.010556,0.414103,True
2,mild,1260,1.0,-0.890671,2.717381,0.012591,0.725776,True
3,mild,1260,1.5,-0.891316,2.725706,0.011009,0.896742,True
4,mild,1260,2.0,-0.887800,2.707752,0.013692,0.969330,True
5,moderate,1260,0.0,-1.730912,7.494639,0.011358,0.011270,False
6,moderate,1260,0.5,-1.733837,7.529401,0.011753,0.416808,True
7,moderate,1260,1.0,-1.729185,7.495455,0.014353,0.724535,True
8,moderate,1260,1.5,-1.732310,7.510545,0.011557,0.889744,True
9,moderate,1260,2.0,-1.731574,7.496714,0.011386,0.960428,True


# Precision and Recall of PSR

In [2]:
# SR0 = 0, SR1 ∈ {0.5, 1.0, 1.5, 2.0}

SR0_annual = 0.0
SR0_daily  = 0.0
SR1_annual_list = [0.5, 1.0, 1.5, 2.0]
SR1_daily_list  = [s/np.sqrt(252) for s in SR1_annual_list]

def _confusion_metrics(y_true, pvals, alpha=0.05):
    yhat = (pvals < alpha)
    TP = int(((y_true==1)&(yhat)).sum())
    FP = int(((y_true==0)&(yhat)).sum())
    TN = int(((y_true==0)&(~yhat)).sum())
    FN = int(((y_true==1)&(~yhat)).sum())
    prec = TP/(TP+FP) if (TP+FP)>0 else np.nan
    rec  = TP/(TP+FN) if (TP+FN)>0 else np.nan
    f1   = (2*prec*rec)/(prec+rec) if (prec>0 and rec>0) else (0.0 if (prec==0 or rec==0) else np.nan)
    return prec, rec, f1

rows = []
for cfg in configs:
    for SR1_daily, SR1_annual in zip(SR1_daily_list, SR1_annual_list):
        # Null: SR = 0 ; Alternative: SR = SR1 (annual)
        X0 = gen_with_true_SR0(REPS, T, cfg, SR0=SR0_daily, seed=RSEED)
        X1 = gen_with_true_SR0(REPS, T, cfg, SR0=SR1_daily, seed=RSEED+1)
        y_true = np.r_[np.zeros(len(X0), dtype=int), np.ones(len(X1), dtype=int)]

        # PSR one-sided test (H1: SR > 0)
        p_psr = np.r_[stats.norm.sf(psr_z_T(X0, SR0_daily)),
                      stats.norm.sf(psr_z_T(X1, SR0_daily))]
        prec, rec, f1 = _confusion_metrics(y_true, p_psr, alpha=0.05)

        rows.append({
            "config": cfg[0],
            "SR1_annual": SR1_annual,
            "PSR_precision": prec,
            "PSR_recall": rec,
            "PSR_F1": f1
        })

psr_table = pd.DataFrame(rows).sort_values(
    ["config","SR1_annual"]
).set_index(["config","SR1_annual"]).round(4)
psr_table.to_csv('appendix_2.csv')
psr_table


PSR_precision  PSR_recall  PSR_F1
config   SR1_annual                                   
mild     0.5                0.8466      0.3047  0.4481
         1.0                0.9288      0.7199  0.8111
         1.5                0.9450      0.9488  0.9469
         2.0                0.9475      0.9971  0.9717
moderate 0.5                0.8396      0.3051  0.4476
         1.0                0.9250      0.7186  0.8088
         1.5                0.9419      0.9452  0.9435
         2.0                0.9447      0.9950  0.9692
severe   0.5                0.8388      0.3112  0.4540
         1.0                0.9227      0.7139  0.8050
         1.5                0.9405      0.9446  0.9425
         2.0                0.9433      0.9958  0.9689